## Identifying and resolving conflicts in a batch of VI files

ELGs - part x is sets xx to xx

There are xxx visual inspections of a total of xxx unique objects
xx inspections per object.

xx conflicts

In [1]:
save_filename=

SyntaxError: invalid syntax (<ipython-input-1-0414865cf7b9>, line 1)

In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

In [3]:
pd.set_option('display.max_rows', 20)

In [4]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [5]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(11,16):
    pattern = "desi*_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

vi_files

desi*_11_*.csv
desi*_12_*.csv
desi*_13_*.csv
desi*_14_*.csv
desi*_15_*.csv


['desi-vi_SV0_ELG_tile67230_night20200315_11_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_11_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_11_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_BLH.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_BJW.csv']

In [6]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_11_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_11_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_BLH.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_BJW.csv
There are 750 visual inspections of a total of 250 unique objects
That gives 3.0 inspections per object.


In [7]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [8]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [9]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
750


### Adding a bunch of useful columns

In [ ]:
vi = VI.prep_new_columns(vi)

Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [18]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [19]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185754028118049 35185754028118319 35185754032309199 35185754032309750
 35185754032310573 35185754032310703 35185754032313260 35185754036502670
 35185754036503154 35185754036503775 35185754036503778 35185754036503929
 35185754036503964 35185754036504236 35185754036504971 35185754036505256
 35185754036505827 35185754036506000 35185754036506049 35185754040697856
 35185754040698036 35185754040698310 35185754040699512 35185754040699533
 35185754040699635 35185754040700128 35185754040700397 35185754040700421
 35185754040700555 35185754040700578 35185754040700862 35185754040701930
 35185754044891804 35185754044892011 35185754044892719 35185754044893397
 35185754044893490 35185754044894142 35185754044894357 35185754044894937
 35185754044895452 35185754044895763 35185754049088211 35185754053281072
 35185754053282143 35185754053282660 35185754053283933 35185754053284109
 35185754053284658 35185754057474179 35185754057475373 35185754057475889
 35185754057477287 35

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [21]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [22]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [23]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=True)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
11,35185754028118049,GALAXY,1.528,BJW,0,--,--,--,[O II] among skylines not convincing,1.528,GALAXY,1.0,2,1.528,0.0,1|1| | | | | | | | | | | | | | | | | | |[|O| |...,3,none
61,35185754028118049,GALAXY,1.528,CGS,1,--,--,--,--,1.528,GALAXY,1.0,2,1.528,0.0,1|1| | | | | | | | | | | | | | | | | | |[|O| |...,3,none
111,35185754028118049,GALAXY,1.528,vrk,2,R,--,--,doublet in skyline noise region + VI redshift...,1.528,GALAXY,1.0,2,1.528,0.0,1|1| | | | | | | | | | | | | | | | | | |[|O| |...,3,none


### and so on...

We should now recompute the conflicts, and not find any.

In [260]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [261]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191335094849374]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [265]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [266]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [267]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [268]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [ ]:
print(VI_dir+save_filename)

In [269]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [270]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [271]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'DELTACHI2', 'FIBER',
       'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z'],
      dtype='object')

In [272]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35191325322120560,0.1012,GALAXY,0.1012,GALAXY,4.000000,13,2248.126427,500,8.915575,13.690263,18.567017,3.849107,5.910476,8.015910
1,35191325326313329,0.2555,GALAXY,0.2555,GALAXY,4.000000,13,597.103415,471,3.061106,6.622557,11.148005,1.005650,2.175676,3.662399
2,35191325326313441,0.1513,GALAXY,0.1513,GALAXY,3.923077,13,402.013473,459,8.835087,15.250219,22.150007,1.184792,2.045066,2.970333
3,35191325326313842,0.1255,GALAXY,0.1255,GALAXY,4.000000,12,1171.531194,356,7.692234,12.353596,16.146137,2.066740,3.319149,4.338124
4,35191325326313983,0.0213,GALAXY,0.0213,GALAXY,4.000000,13,1233.152205,398,9.700712,11.533047,10.710887,1.310380,1.557893,1.446835
5,35191325326314275,0.5843,GALAXY,0.5843,GALAXY,3.769231,13,215.043012,191,3.716881,10.441963,23.580444,0.818709,2.300028,5.194012
6,35191325326314402,0.4220,GALAXY,0.4220,GALAXY,4.000000,13,1241.922597,397,4.036667,10.297829,18.527788,1.408013,3.591943,6.462602
7,35191325326314538,0.4593,GALAXY,0.4593,GALAXY,3.000000,13,360.383502,363,1.326949,6.981154,19.884447,0.448803,2.361180,6.725359
8,35191325326314565,0.2785,GALAXY,0.2785,GALAXY,3.769231,13,487.159330,192,3.047705,12.858685,27.577822,1.625618,6.858705,14.709757
9,35191325326314860,0.3363,GALAXY,0.3363,GALAXY,3.000000,13,284.628837,184,2.184791,9.017674,19.233717,1.025388,4.232265,9.026961


In [ ]:
merged_file.loc[9]['merger comment']

In [ ]:
merged_file.loc[9]['all VI comments']

In [273]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/BGS/'